<a href="https://colab.research.google.com/github/bdemchak/cytoscape-jupyter/blob/main/gangsu/basic%20protocol%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a reproduction of the [Biological Network Exploration with Cytoscape 3](https://pubmed.ncbi.nlm.nih.gov/25199793/) Basic Protocol 1, which loads an s. cervesiae network, filters out unneeded nodes, lays out the resulting network, creates clusters of similar nodes and then performs an enrichment calculation on one cluster.

Note that this workflow executes in a Jupyter Notebook running on a cloud server (e.g., Google Colab) and communicates with a copy of Cytoscape running on your workstation. For version of this workflow that runs on the same workstation as Cytoscape, see [here](https://github.com/cytoscape/py4cytoscape/tree/master/tests).

---
# Setup data files, py4cytoscape and Cytoscape connection
**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

This workflow requires two files that are located in cloud storage:

* BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab (network file)
* GDS112_full.soft (annotation file)

Both files reside in a Dropbox folder, and they are downloaded by this workflow as needed.

## Setup: Fetch latest py4cytoscape




**Note that you can fetch from the latest Github unreleased version by setting _PY4CYTOSCAPE to 'git+https://github.com/cytoscape/py4cytoscape' immediately before the exec() call. To fetch a particular branch, add '@' to the end (e.g., 'git+https://github.com/cytoscape/py4cytoscape@0.0.12').**

To load the default (PyPI) py4cytoscape version, do not set _PY4CYTOSCAPE at all.

In [1]:
_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@1.4.0' # optional
import requests

exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)

IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/cytoscape/py4cytoscape (to revision 1.4.0) to /tmp/pip-req-build-zwot55qx


  Running command git clone -q https://github.com/cytoscape/py4cytoscape /tmp/pip-req-build-zwot55qx
  Running command git checkout -b 1.4.0 --track origin/1.4.0
  Switched to a new branch '1.4.0'
  Branch '1.4.0' set up to track remote branch '1.4.0' from 'origin'.


  Created wheel for py4cytoscape: filename=py4cytoscape-1.4.0-py3-none-any.whl size=168071 sha256=be600b5f6a67dd41b95f359fecd170d5dbfbece2690101a3ed0f2e5081b3c64d
  Stored in directory: /tmp/pip-ephem-wheel-cache-odvxhgkx/wheels/52/d9/84/a2c7632ccc1801285f9c2a7ff92d7b99a0adcdac3a9c8a8c3a
  Created wheel for python-igraph: filename=python_igraph-0.9.11-py3-none-any.whl size=9074 sha256=3bad89f093589d815b20fa336cfd7266f9f0b221ae577c37b64eada53c1f89c3
  Stored in directory: /root/.cache/pip/wheels/7d/e4/0c/0c83f70bd0b99ce8aea47a21f8e52502169e7dd17808d12f30
Successfully built py4cytoscape python-igraph
NumExpr defaulting to 2 threads.
Loading Javascript client ... 06370b12-1c7a-402a-ba7a-4c1164ac64da on https://jupyter-bridge.cytoscape.org
ADVICE: WHEN RUNNING UNDER COLAB, DO NOT RE-RUN THIS CELL WITHOUT MANUALLY EXECUTING Runtime | Factory Reset Runtime FROM THE COLAB MENU FIRST.


<IPython.core.display.Javascript object>

## Setup: Sanity test to verify Cytoscape connection

By now, the connection to Cytoscape should be up and available. To verify this, try a simple operation that doesn't alter the state of Cytoscape.

In [2]:
p4c.cytoscape_version_info()


{'apiVersion': 'v1',
 'automationAPIVersion': '1.5.0',
 'cytoscapeVersion': '3.9.1',
 'jupyterBridgeVersion': '0.0.2',
 'py4cytoscapeVersion': '1.4.0'}

## Setup: Notebook data files
Create the 'output' directory, which will be used to store files uploaded from Cytoscape.

This is a good place to prepare any other system resources that might be needed by downstream Notebook cells.





In [3]:
!rm -r output/
!ls -l 
OUTPUT_DIR = 'output/'

rm: cannot remove 'output/': No such file or directory
total 8
drwxr-xr-x 2 root root 4096 Jun 15 00:51 logs
drwxr-xr-x 1 root root 4096 Jun  1 13:50 sample_data


## Setup: Import source data files
The network and annotation files are in a Dropbox folder, and this cell downloads them into the default Sandbox from where Cytoscape will access them.

The files could just as well have been on any cloud resource, including Google Drive, Github, Microsoft OneDrive or a private web site. Note that in this case, the network file was so large that it could not be saved on GitHub, so Dropbox was a handy alternative.

*An alternative would be to load the files into this Notebook's file system (or create them there) and then download those files to the Sandbox. Loading them into the Notebook file system would require the use of Notebook "!" commands (e.g., !wget).*


**Sandboxing is explained in https://py4cytoscape.readthedocs.io/en/latest/concepts.html#sandboxing**

In [4]:
p4c.sandbox_set(None) # Revert to default sandbox in case some other workflow selected a different one

res_mitab = p4c.sandbox_url_to("https://www.dropbox.com/s/8wc8o897tsxewt1/BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab?dl=0", "BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab")
print(f'Network file BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab has {res_mitab["fileByteCount"]} bytes')

Network file BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab has 166981992 bytes


---
# Load the Protein-protein Interaction Network into Cytoscape
The network is contained in the s. cerevisiae MITAB file.

Note that in this cell, the `import_network_from_file()` function (incorrectly) throws an exception in pre-3.10.0 Cytoscape. To ignore the exception, we enclose it in a try/except block.

**Note:** Once the CYTOSCAPE-12772 issue is solved, we can remove the try/except block.


In [5]:
from requests import HTTPError
p4c.close_session(False)

try:
  p4c.import_network_from_file('BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab')
except:  
  pass
if p4c.get_network_count() != 1:
  raise Exception('Failed to load network')
net_suid = p4c.get_network_suid()
print(f'Network identifier: {net_suid}')



In commands_post(): {'status': 500, 'type': 'urn:cytoscape:ci:cyrest-core:v1:handle-json-command:errors:3', 'message': 'Task returned invalid json.', 'link': 'file:/C:/Users/CyDeveloper/CytoscapeConfiguration/3/framework-cytoscape.log'}


Network identifier: 63327


---
# Import the gene expression data
The expression data is downloaded and merged into the network's node attribute table.

---
*Tip:* This cell shows how to create code that works around changes in Cytoscape capabilities. 

In this case, starting with Cytoscape 3.9.0, the `load_table_data_from_file()` function works as expected, so the gene expression data is merged into the node attribute table. 

Prior to 3.9.0, `load_table_data_from_file()` didn't work. As a workaround, we do most of the work in Pandas and then import the dataframe into the node attribute table. After Pandas reads the CSV, we will try to match dataframe Gene ID column to the `name` column in the Cytoscape node attribute table. To do this, we must explicitly set the Gene ID as a string (even though it's originally parsed as a number) because Cytoscape's `name` column is already a string. 



In [6]:
if p4c.check_supported_versions(cytoscape='3.9') is None:
  # Load file directly into Sandbox so Cytoscape can import it
  res_soft = p4c.sandbox_url_to("https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0", "GDS112_full.soft")
  print(f'Annotation file GDS112_full.soft has {res_soft["fileByteCount"]} bytes')

  res = p4c.load_table_data_from_file('GDS112_full.soft', start_load_row=83, data_key_column_index=10, delimiters='\t')
  print(f'Load result contains table identifiers: {res["mappedTables"]}')
else:
  # Load file into Notebook file system so Python can import it, tweak it, and download to Cytoscape
  !wget -q --no-check-certificate https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0
  !mv GDS112_full.soft?dl=0 GDS112_full.soft

  import pandas as df
  GDS112_full = df.read_csv('GDS112_full.soft', skiprows=82, sep='\t')
  GDS112_full.dropna(subset=['Gene ID'], inplace=True)
  GDS112_full['Gene ID'] = df.to_numeric(GDS112_full['Gene ID'], downcast='integer')
  GDS112_full = GDS112_full.astype({'Gene ID': 'string'})
  print(GDS112_full.dtypes)
  print(GDS112_full)
  p4c.load_table_data(GDS112_full, data_key_column='Gene ID')

  import os
  os.remove('GDS112_full.soft')


Annotation file GDS112_full.soft has 5536880 bytes
Load result contains table identifiers: [63298, 63336]


---
# Filter the Network with the Genes that have Expression Data
For this, we assume that if a node has no *Gene symbol*, it also has no expression data. 

The filter compares each node's *Gene symbol* attribute to a regular expression. If there is a match, the gene is selected; for no match, the gene isn't selected.

In [7]:
res = p4c.create_column_filter('SymbolOK', 'Gene symbol', '[A-Z0-9]*', 'REGEX')
print(f'Nodes selected: {len(res["nodes"])}')

No edges selected.
Nodes selected: 5514


---
# Create a New Network with the Selected Subset
Create a subnetwork containing only nodes selected by the filter (i.e., having a *Gene symbol* value, which implies that expression data is present for that node).

This could take several minutes.

At the end, you should see a view containing all nodes laid out. 

If you see only a single rectangle, it could be that your Cytoscape is set to operate with a small stack size. To increase the stack:

1. terminate Cytoscape

2. a) upgrade Cytoscape to 3.9.0 or later 

  ... or b) use a text editor to add -Xss5M to the cytoscape.vmoptions file in your Cytoscape program directory

3. restart Cytoscape

4. re-run this workflow

In [8]:
new_suid = p4c.create_subnetwork()
print(f'New network identifier: {new_suid}')

New network identifier: 1451225


## Get rid of the original network, which isn't needed anymore

In [9]:
p4c.delete_network(net_suid)
net_suid = new_suid

---
# Identify Network Modules
The overall strategy is to find clusters of nodes that share some common attribute. In this case, we use expression data values. Specifically:

* Load Cytoscape's clusterMaker2 app
* Use clusterMaker2 to create a dendogram showing a hierarchy of similar network modules


## Install clusterMaker2 if it hasn't already been installed

In [10]:
p4c.install_app('clusterMaker2')

{}


{}

## Identify network modules
Create a hierarchic clustering of similar nodes based on the expression data columns. Cytoscape renders the hierarchy as a dendogram.

*Tip:* Cytoscape's dendogram window can be used to manually explore module similarity. 

In [11]:
dendo_clustering = p4c.commands_post('cluster hierarchical showUI=true clusterAttributes=false nodeAttributeList="GSM1029,GSM1030,GSM1032,GSM1033,GSM1034"')

# dendo_clustering is a dictionary [{nodeOrder: [{nodeName: xxx, suid: sss}, ...]}
#                                   {nodeTree: [{name: ggg, left: lll, right: rrr}]}]
# where nodeOrder is a mapping between a leaf node name xxx and the suid sss of a network node,
# and nodeTree is a tree where the left node lll and right node rrr can be leaf nodes xxx or
# internal nodes ggg. 

---
# Perform an enrichment analysis using the gprofiler package

Use a package commonly available in PyPI to calculate functional enrichment for nodes similar to a node in which we may be interested. It's an example of how Cytoscape can work together with Python-based libraries to achieve a useful result.

In this case, we choose HBT1 (entrez-gene ID 851303).

1. Find SUID of network's HBT1 node

1. Find a set of nodes similar to HBT1 by collecting nodes nearby in the tree

1. Use each node's SUID to look up its entrez-gene ID

1. Pass the set of entrez-gene IDs to gprofiler as an enrichment query




## Find HBD1 in the similarity tree

In [12]:
node_suid = p4c.node_name_to_node_suid('851303')[0] # Use entrez-gene ID to get SUID for HBT1

## Collect set of SUIDs representing 85 similar nodes
Note that we use custom functions to parse and traverse the dendogram's similarity tree.

We use *wget* to copy the functions into a local directory so we can them import them and then call them.

In [13]:
!wget -c https://raw.githubusercontent.com/cytoscape/py4cytoscape/master/tests/Notebooks/parse_dendogram.py
import parse_dendogram as pde # Use custom functions to decode dendogram tree

node_order = dendo_clustering[0]['nodeOrder']
node_tree = dendo_clustering[0]['nodeTree']

node_bag = pde.create_node_bag(node_order, node_tree)
similar_nodes = list(pde.find_node_set(node_suid, 85, node_order, node_bag))

--2022-06-15 01:05:34--  https://raw.githubusercontent.com/cytoscape/py4cytoscape/master/tests/Notebooks/parse_dendogram.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1852 (1.8K) [text/plain]
Saving to: ‘parse_dendogram.py’

parse_dendogram.py  100%[===================>]   1.81K  --.-KB/s    in 0s      

2022-06-15 01:05:34 (21.7 MB/s) - ‘parse_dendogram.py’ saved [1852/1852]



## Using SUIDs, query Cytoscape for each node's entrez-gene ID

In [14]:
suid_to_entrez_gene = p4c.get_table_columns(columns='name')['name']
entrez_gene_query = [int(suid_to_entrez_gene[suid])  for suid in similar_nodes]

print(entrez_gene_query)

[850295, 855363, 854742, 853614, 854197, 854212, 853766, 851321, 855878, 851303, 851295, 854244, 850627, 854125, 854214, 852902, 854000, 851541, 850425, 856616, 853693, 855789, 855737, 855227, 854206, 854098, 850412, 853405, 856543, 854297, 854247, 851649, 852695, 853613, 852436, 853217, 851596, 854240, 855738, 851934, 855669, 854898, 853159, 855020, 851380, 854188, 851433, 851239, 851474, 850963, 855774, 854302, 854357, 851581, 853758, 852146, 854062, 855562, 850318, 851013, 851625, 856855, 852342, 855788, 852543, 855836, 852343, 852291, 853238, 851582, 852064, 853920, 856417, 856018, 853536, 853169, 851347, 854108, 854856, 852721, 852818, 850597, 853309, 854204, 851445]


## Install gprofiler package if it's not already installed

In [15]:
!pip install gprofiler-official
from gprofiler import GProfiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Use entrez-gene IDs to query gprofiler for GO functional enrichment

In [16]:
gp = GProfiler(user_agent='py4cytoscape', return_dataframe=True)
gp.profile(organism='scerevisiae', query=entrez_gene_query)

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents
0,GO:CC,GO:0005576,extracellular region,0.000072,True,"""The space external to the outermost structure...",127,82,11,6567,0.134146,0.086614,query_1,[GO:0110165]
1,KEGG,KEGG:01200,Carbon metabolism,0.000225,True,Carbon metabolism,112,35,10,2091,0.285714,0.089286,query_1,[KEGG:00000]
2,GO:MF,GO:0003824,catalytic activity,0.000407,True,"""Catalysis of a biochemical reaction at physio...",2333,83,51,6549,0.614458,0.021860,query_1,[GO:0003674]
3,KEGG,KEGG:00260,"Glycine, serine and threonine metabolism",0.003018,True,"Glycine, serine and threonine metabolism",30,35,5,2091,0.142857,0.166667,query_1,[KEGG:00000]
4,GO:BP,GO:0009063,cellular amino acid catabolic process,0.008424,True,"""The chemical reactions and pathways resulting...",63,83,7,6540,0.084337,0.111111,query_1,"[GO:0006520, GO:0046395, GO:1901565]"
5,KEGG,KEGG:00650,Butanoate metabolism,0.009858,True,Butanoate metabolism,9,35,3,2091,0.085714,0.333333,query_1,[KEGG:00000]
6,GO:CC,GO:0005622,intracellular anatomical structure,0.010007,True,"""A component of a cell contained within (but n...",5216,82,78,6567,0.951220,0.014954,query_1,[GO:0110165]
7,GO:CC,GO:0005737,cytoplasm,0.010971,True,"""The contents of a cell excluding the plasma m...",4435,82,71,6567,0.865854,0.016009,query_1,"[GO:0005622, GO:0110165]"
8,GO:BP,GO:0043436,oxoacid metabolic process,0.014193,True,"""The chemical reactions and pathways involving...",431,83,17,6540,0.204819,0.039443,query_1,[GO:0006082]
9,GO:BP,GO:0006082,organic acid metabolic process,0.016001,True,"""The chemical reactions and pathways involving...",435,83,17,6540,0.204819,0.039080,query_1,"[GO:0044237, GO:0044281, GO:0071704]"
